In [3]:
import os

from accelerate.commands.config.update import description
from click import prompt
from langchain.chains.sequential import SequentialChain
from langchain.chains.summarize.refine_prompts import prompt_template
from langchain_cohere import ChatCohere
from langchain.prompts import ChatPromptTemplate
import os
from dotenv import load_dotenv
from pydantic.experimental.pipeline import transform
from sympy.physics.units import temperature

load_dotenv()


C:\Users\USER\.virtualenvs\The_Rocket-q1L1raes\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [4]:

api_key = os.getenv("COHERE_API_KEY")
cohere_chat = ChatCohere(
    api_key=api_key,
    temperature=0.0, 
    max_tokens=100,
)


system_prompt_template = """
For the following text, extract the following information:

Text:
{text}

1. What is the main idea of the text?
2. What is the main argument of the text?
3. What are the main points of the text?
4. What are the main examples of the text?

Please format the output in a JSON-like structure:
- main_idea: The main idea of the text
- main_argument: The main argument of the text
- main_points: The main points of the text
- main_examples: The main examples of the text
"""


review_text="""The main idea of the text is that the world is a complex place and that we need to be able to understand it in order to make informed decisions. The main argument of the text is that we need to be able to think critically about the information that we are presented with in order to make sense of it. The main points of the text are that the world is a complex place, that we need to be able to think critically about the information that we are presented with, and that we need to be able to make informed decisions. The main examples of the text are that we need to be able to understand the world in order to make informed decisions, that we need to be able to think critically about the information that we are presented with, and that we need to be able to make sense of the information that we are presented with."""

chat_prompt_template = ChatPromptTemplate.from_template(system_prompt_template)

In [5]:
messages = chat_prompt_template.format_messages(text=review_text)
response = cohere_chat(messages)

print(response.content)

C:\Users\USER\AppData\Local\Temp\ipykernel_6188\327023887.py:2: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = cohere_chat(messages)


{
  "main_idea": "The world is a complex place and understanding it is crucial for making informed decisions.",
  "main_argument": "Critical thinking is essential to process and comprehend the information we encounter.",
  "main_points": [
    "The world's complexity",
    "The need for critical thinking",
    "The importance of informed decision-making"
  ],
  "main_examples": [
    "Understanding the world for informed decision-making",
    "Applying critical thinking to presented information",
    "Making sense of information"
  ]
}


In [6]:
main_ideas = response.content.get('main_idea')

AttributeError: 'str' object has no attribute 'get'

In [122]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

main_idea_schema = ResponseSchema(
    name="main idea",
    description="The main idea of the text",
)

main_argument_schema = ResponseSchema(
    name="main argument",
    description="The main argument of the text",
)

main_points_schema = ResponseSchema(
    name="main points",
    description="The main points of the text",
)


response_schemas = [
    main_idea_schema,
    main_argument_schema,
    main_points_schema,
]


output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [127]:
format_instructions=output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"main idea": string  // The main idea of the text
	"main argument": string  // The main argument of the text
	"main points": string  // The main points of the text
}
```


In [128]:
system_prompt_2="""
For the following text, extract the following information:
        
Text:
{text}

1. What is the main idea of the text?
2. What is the main argument of the text?
3. What are the main points of the text?
4. What are the main examples of the text?

{format_instructions}
"""

In [130]:
chat_prompt_template_2 = ChatPromptTemplate.from_template(system_prompt_2)

message_2 = chat_prompt_template_2.format_messages(text=review_text, format_instructions=format_instructions)

In [142]:
response_2=cohere_chat(message_2)
print(response_2.content)

```json
{
	"main idea": "Understanding the complexity of the world and the importance of critical thinking",
	"main argument": "The necessity of critical thinking to make sense of information and navigate the world",
	"main points": "The world's complexity, the need for critical thinking, and the ability to make informed decisions",
	"main examples": "Understanding the world for decision-making, thinking critically about information, and making sense of presented information"
}
```


In [145]:
structured_output = output_parser.parse(response_2.content)
print(type(structured_output))

<class 'dict'>


In [146]:
print(structured_output.get('main idea')) 

Understanding the complexity of the world and the importance of critical thinking


## SimpleSequentialChain

In [21]:
from langchain.chains import SequentialChain, SimpleSequentialChain
from langchain.chains.transform import TransformChain

In [27]:
llm = ChatCohere(temperature=0.9, max_tokens=100)
    

first_prompt = ChatPromptTemplate.from_template("What is the best name to describe a company that makes {product}?")

    
def generate_company_name(inputs):
    product = inputs['product']
    # Here you would call your LLM or any logic to generate a company name
    return {"company_name": f"Best {product} Co."}  # Example output


chain_1 = TransformChain(input_variables=["product"], output_variables=["company_name"], transform=generate_company_name)


In [33]:
# second template

second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)

def generate_description(inputs):
    company_name = inputs['company_name']
    return {"description": f"{company_name} specializes in eco-friendly products."}

chain_2 = TransformChain(input_variables=["company_name"], output_variables=["description"], transform=generate_description)


In [48]:
# third template

third_prompt = ChatPromptTemplate.from_template(
        "What language is the following review:\n\n{Review}"
)

def get_review(inputs):
    review = inputs['Review']
    return f"review: {review}"
chain_3 = TransformChain(input_variables=["Review"], output_variables=["language"], transform=get_review)

In [64]:
# fourth template
fourth_prompt = ChatPromptTemplate.from_template(
    "What is the main idea of the following text: \n\n{text}"
)

def get_main_idea(inputs):
    text, language = inputs['text', 'language']
    messages = chat_prompt_template.format_messages(text=text, language=language)
    response = cohere_chat(messages)
    structured_output = output_parser.parse(response.content)
    return structured_output

chain_4 = TransformChain(input_variables=["text"], output_variables=["main_idea"], transform=get_main_idea)

In [65]:
from langchain.chains import SequentialChain

seq_chain = SequentialChain(chains=[chain_1, chain_2, chain_3, chain_4], output_variables=["company_name"], input_variables=["Review", "product", "text"], verbose=True)

In [66]:
# Example usage (make sure to provide appropriate inputs)
result = seq_chain({"product": "eco-friendly packaging", 
                        "Review": "This product is amazing and I would recommend it to everyone!",
                        "text": "The main idea of the text is that the world is a complex place and that we need to be able to understand it in order to make informed decisions."
                        })
print(result)




> Entering new SequentialChain chain...


ValueError: Missing some output keys: {'language'}